In [ ]:
# 本文讲述了如何通过Langchain的MultiQueryRetriever、LongContextReorder和ContextualCompression技术改善RAG效果，
# 减少无关文档干扰，提升大型语言模型的提问与总结能力。

## 一、Multi Query Retriever[1]

In [ ]:
# build a sample vectorDB

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
import StrOutputParser
import ChatPromptTemplate
import os

os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# load blog post
loader = WebBaseLoader("https://www.investopedia.com/terms/c/capital-asset-pricing-model.asp")
data = loader.load()

llm = ChatOpenAI()

# split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# vectorDB
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(splits, embeddings)

In [ ]:
# prepare multi query retriever
from langchain.retrievers import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

question = "What is the capital of France?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(), llm=llm)

In [ ]:
# setup QnA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

qa_system_prompt = """
    Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    {context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "{question}"),
    ]
)

def format_docs (docs):
    doc_strings = [doc.page_content for doc in docs]
    return "\n\n".join(doc_strings)

rag_chain = (
    {
        "context": retriever_from_llm | format_docs,
        "question": RunnablePassthrough(),
    }
    | qa_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What did the author do growing up?")

## 二、Long Context Reorder[2]

In [ ]:
# 长上下文重排序非常适合于我们想要从矢量数据库返回10个以上文档的情况。

# QnA Part and Reordering
from langchain_community.document_transformers import (
    LongContextReorder,
)

def format_docs(docs):
    # call the reordering function in here
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    doc_strings = [doc.page_content for doc in reordered_docs]
    return "\n\n".join(doc_strings)

rag_chain = (
    {
        "context": vectordb.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    } 
    | qa_prompt
    | ChatOpenAI(temperature=0) 
    | StrOutputParser()
)

rag_chain.invoke("What is the capital of France?")

## 三、Contextual Compression[4]

In [ ]:
# 每个块长度很长，且包含有不相干的信息；使用LLM从检索到的文档中删除那些不相关的段落。
# contextual compression setup
from langchain.retrievers.document_compressors import DocumentCompressorPipeline, EmbeddingsFilter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers import ContextualCompressionRetriever

In [ ]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings,similarity_threshold=0.9)
pipeline_compressor = DocumentCompressorPipeline(transformers=[splitter, redundant_filter, relevant_filter])
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor,
    base_retriever=vectordb.as_retriever()
)

In [ ]:
def format_docs(docs):
    doc_strings = [doc.page_content for doc in docs]
    return "\n\n".join(doc_strings)

rag_chain = (
    {
        "context": compression_retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | qa_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What did the author do growing up?")